In [88]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pyodbc

# Collect GKM standings in PGE Ekstraliga across 2015-2024

In [57]:
base_url = 'https://ekstraliga.pl/se/tabela/pgee/'
suffixes = np.arange(2015,2025)
frames = []

for suffix in suffixes:
    url = f"{base_url}{suffix}"
    response = requests.get(url)

    if response.status_code == 200:
        tables = pd.read_html(response.text)
        df = tables[0]
        df.insert(0,"Sezon",suffix)
        frames.append(df)
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
df_teams_stand = pd.concat(frames, ignore_index=True)

In [58]:
df_teams_stand.head(5)

,Sezon,Unnamed: 0,Unnamed: 1,Unnamed: 2,M,Z,R,P,B,PKT,+/-
0,2015,NaN,1.0,FOGO Unia LesznoLES,14.0,12.0,0.0,2.0,5.0,29.0,89.0
1,2015,NaN,2.0,Unia TarnówTAR,14.0,8.0,1.0,5.0,4.0,21.0,17.0
2,2015,NaN,3.0,BETARD Sparta WrocławWRO,14.0,6.0,1.0,7.0,5.0,18.0,68.0
3,2015,NaN,4.0,KS ToruńTOR,14.0,6.0,2.0,6.0,3.0,17.0,27.0
4,2015,NaN,5.0,SPAR Falubaz Zielona GóraZIE,14.0,6.0,1.0,7.0,3.0,16.0,-1.0


In [59]:
df_teams_stand.drop(df_teams_stand.columns[1], axis=1, inplace=True)

In [60]:
df_teams_stand.rename(columns={'Unnamed: 1':'Pozycja','Unnamed: 2': 'Klub',
                               'M':'Mecze','Z':'Zwyciestwa','R':'Remisy','P':'Porazki',
                               'B':'Bonusy','PKT':'Punkty','+/-':'Male_Punkty'}, inplace=True)

In [61]:
df_teams_stand

,Sezon,Pozycja,Klub,Mecze,Zwyciestwa,Remisy,Porazki,Bonusy,Punkty,Male_Punkty
0,2015,1.0,FOGO Unia LesznoLES,14.0,12.0,0.0,2.0,5.0,29.0,89.0
1,2015,2.0,Unia TarnówTAR,14.0,8.0,1.0,5.0,4.0,21.0,17.0
2,2015,3.0,BETARD Sparta WrocławWRO,14.0,6.0,1.0,7.0,5.0,18.0,68.0
3,2015,4.0,KS ToruńTOR,14.0,6.0,2.0,6.0,3.0,17.0,27.0
4,2015,5.0,SPAR Falubaz Zielona GóraZIE,14.0,6.0,1.0,7.0,3.0,16.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
85,2024,5.0,ZOOLESZCZ GKM GrudziądzGRU,14.0,6.0,0.0,8.0,2.0,14.0,-58.0
86,2024,6.0,NOVYHOTEL FALUBAZ Zielona Góra (B)ZIE (B),14.0,4.0,2.0,8.0,3.0,13.0,-33.0
87,2024,7.0,KRONO-PLAST WŁÓKNIARZ CzęstochowaCZE,14.0,4.0,2.0,8.0,2.0,12.0,-50.0
88,2024,8.0,FOGO UNIA LesznoLES,14.0,5.0,1.0,8.0,0.0,11.0,-87.0


In [62]:
df_teams_stand['Klub'].isna().sum()

10

In [63]:
df_teams_stand = df_teams_stand.dropna(subset=["Klub"])

In [64]:
df_teams_stand

,Sezon,Pozycja,Klub,Mecze,Zwyciestwa,Remisy,Porazki,Bonusy,Punkty,Male_Punkty
0,2015,1.0,FOGO Unia LesznoLES,14.0,12.0,0.0,2.0,5.0,29.0,89.0
1,2015,2.0,Unia TarnówTAR,14.0,8.0,1.0,5.0,4.0,21.0,17.0
2,2015,3.0,BETARD Sparta WrocławWRO,14.0,6.0,1.0,7.0,5.0,18.0,68.0
3,2015,4.0,KS ToruńTOR,14.0,6.0,2.0,6.0,3.0,17.0,27.0
4,2015,5.0,SPAR Falubaz Zielona GóraZIE,14.0,6.0,1.0,7.0,3.0,16.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
84,2024,4.0,KS APATOR ToruńTOR,14.0,6.0,0.0,8.0,3.0,15.0,-7.0
85,2024,5.0,ZOOLESZCZ GKM GrudziądzGRU,14.0,6.0,0.0,8.0,2.0,14.0,-58.0
86,2024,6.0,NOVYHOTEL FALUBAZ Zielona Góra (B)ZIE (B),14.0,4.0,2.0,8.0,3.0,13.0,-33.0
87,2024,7.0,KRONO-PLAST WŁÓKNIARZ CzęstochowaCZE,14.0,4.0,2.0,8.0,2.0,12.0,-50.0


In [65]:
df_teams_stand = df_teams_stand[df_teams_stand['Klub'].str.contains('GKM')].reset_index(drop=True)

In [66]:
df_teams_stand

,Sezon,Pozycja,Klub,Mecze,Zwyciestwa,Remisy,Porazki,Bonusy,Punkty,Male_Punkty
0,2015,8.0,MRGARDEN GKM Grudziądz (B)GRU (B),14.0,5.0,0.0,9.0,0.0,10.0,-184.0
1,2016,5.0,MRGARDEN GKM GrudziądzGRU,14.0,7.0,0.0,7.0,2.0,16.0,-18.0
2,2017,6.0,MRGARDEN GKM GrudziądzGRU,14.0,4.0,1.0,9.0,1.0,10.0,-63.0
3,2018,6.0,MRGARDEN GKM GrudziądzGRU,14.0,4.0,2.0,8.0,2.0,12.0,-86.0
4,2019,5.0,MRGARDEN GKM GrudziądzGRU,14.0,6.0,1.0,7.0,3.0,16.0,-2.0
5,2020,7.0,MRGARDEN GKM GrudziądzGRU,14.0,4.0,1.0,9.0,1.0,10.0,-102.0
6,2021,7.0,ZOOLESZCZ DPV LOGISTIC GKM GrudziądzGRU,14.0,2.0,3.0,9.0,0.0,7.0,-158.0
7,2022,7.0,ZOOLESZCZ GKM GrudziądzGRU,14.0,5.0,0.0,9.0,2.0,12.0,-136.0
8,2023,7.0,ZOOLESZCZ GKM GrudziądzGRU,14.0,3.0,2.0,9.0,2.0,10.0,-61.0
9,2024,5.0,ZOOLESZCZ GKM GrudziądzGRU,14.0,6.0,0.0,8.0,2.0,14.0,-58.0


In [67]:
df_teams_stand['Klub']='GKM Grudziadz'

In [68]:
df_teams_stand

,Sezon,Pozycja,Klub,Mecze,Zwyciestwa,Remisy,Porazki,Bonusy,Punkty,Male_Punkty
0,2015,8.0,GKM Grudziadz,14.0,5.0,0.0,9.0,0.0,10.0,-184.0
1,2016,5.0,GKM Grudziadz,14.0,7.0,0.0,7.0,2.0,16.0,-18.0
2,2017,6.0,GKM Grudziadz,14.0,4.0,1.0,9.0,1.0,10.0,-63.0
3,2018,6.0,GKM Grudziadz,14.0,4.0,2.0,8.0,2.0,12.0,-86.0
4,2019,5.0,GKM Grudziadz,14.0,6.0,1.0,7.0,3.0,16.0,-2.0
5,2020,7.0,GKM Grudziadz,14.0,4.0,1.0,9.0,1.0,10.0,-102.0
6,2021,7.0,GKM Grudziadz,14.0,2.0,3.0,9.0,0.0,7.0,-158.0
7,2022,7.0,GKM Grudziadz,14.0,5.0,0.0,9.0,2.0,12.0,-136.0
8,2023,7.0,GKM Grudziadz,14.0,3.0,2.0,9.0,2.0,10.0,-61.0
9,2024,5.0,GKM Grudziadz,14.0,6.0,0.0,8.0,2.0,14.0,-58.0


In [69]:
df_teams_stand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sezon        10 non-null     int32  
 1   Pozycja      10 non-null     float64
 2   Klub         10 non-null     object 
 3   Mecze        10 non-null     float64
 4   Zwyciestwa   10 non-null     float64
 5   Remisy       10 non-null     float64
 6   Porazki      10 non-null     float64
 7   Bonusy       10 non-null     float64
 8   Punkty       10 non-null     float64
 9   Male_Punkty  10 non-null     float64
dtypes: float64(8), int32(1), object(1)
memory usage: 888.0+ bytes


In [96]:
cols_to_convert = ['Sezon','Pozycja','Mecze','Zwyciestwa', 'Remisy', 'Porazki', 'Bonusy', 'Punkty', 'Male_Punkty']
df_teams_stand[cols_to_convert] = df_teams_stand[cols_to_convert].astype(int)

In [97]:
df_teams_stand.dtypes

Sezon           int32
Pozycja         int32
Klub           object
Mecze           int32
Zwyciestwa      int32
Remisy          int32
Porazki         int32
Bonusy          int32
Punkty          int32
Male_Punkty     int32
dtype: object

# Insert GKM Standings Data into MS SQL Table

In [99]:
# Database connection parameters
server_name = 'DESKTOP-F00PL4B\SQLEXPRESS01'
database_name = 'PortfolioProject'
table_name = 'GKM_Wyniki_PGE'

# SQLAlchemy engine creation with Windows Authentication
engine = create_engine(f"mssql+pyodbc://@{server_name}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Insert DataFrame into the existing table with column mapping
df_teams_stand.to_sql(name=table_name, con=engine, if_exists='append', index=False)

# Close the database connection
engine.dispose()


# Collect GKM riders data across seasons 2015-2024

In [70]:
base_url = 'https://sportowefakty.wp.pl/zuzel/gkm-grudziadz/statystyki/'
suffixes = np.arange(2015,2025)
frames = []

for suffix in suffixes:
    url = f"{base_url}{suffix}"
    response = requests.get(url)

    if response.status_code == 200:
        tables = pd.read_html(response.text)
        df = tables[0]
        df.insert(0,"Sezon",suffix)
        frames.append(df)
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

df_riders = pd.concat(frames,ignore_index=True)

In [71]:
df_riders

,Sezon,Poz,Zawodnik,Klub,M,B,1,2,3,4,...,U,W,T,Pkt.,Bonus,Razem,Śr/bieg,Śr/mecz,Dom,Wyjazd
0,2015,1,Artiom Łaguta,NaN,14,73,29,15,20,4,...,0,0,1,137,7,144,1973,9786,2147,1821
1,2015,2,Krzysztof Buczkowski,NaN,14,77,16,23,31,6,...,0,0,0,125,13,138,1792,8929,2135,1475
2,2015,3,Tomasz Gollob,NaN,14,72,14,24,17,14,...,0,0,0,107,9,116,1611,7643,2243,943
3,2015,4,Rafał Okoniewski,NaN,14,63,12,16,16,17,...,1,0,0,84,7,91,1444,6000,2000,793
4,2015,5,Daniel Jeleniewski,NaN,11,37,4,5,8,16,...,1,2,1,30,9,39,1054,2727,1308,455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2024,3,Wadim Tarasienko,NaN,14,68,14,26,17,10,...,0,0,1,111,14,125,1838,8539,2000,1656
58,2024,4,Jaimon Lidsey,NaN,14,68,18,18,14,16,...,0,1,1,104,8,112,1647,8000,1649,1645
59,2024,5,Kacper Łobodziński,NaN,14,45,9,5,16,13,...,1,0,1,53,10,63,1400,4077,1458,1333
60,2024,6,Kevin Małkiewicz,NaN,14,39,3,8,11,14,...,0,2,0,36,5,41,1051,2769,1348,625


In [72]:
df_riders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Sezon     62 non-null     int32  
 1   Poz       62 non-null     int64  
 2   Zawodnik  62 non-null     object 
 3   Klub      0 non-null      float64
 4   M         62 non-null     int64  
 5   B         62 non-null     int64  
 6   1         62 non-null     int64  
 7   2         62 non-null     int64  
 8   3         62 non-null     int64  
 9   4         62 non-null     int64  
 10  D         62 non-null     int64  
 11  U         62 non-null     int64  
 12  W         62 non-null     int64  
 13  T         62 non-null     int64  
 14  Pkt.      62 non-null     int64  
 15  Bonus     62 non-null     int64  
 16  Razem     62 non-null     int64  
 17  Śr/bieg   62 non-null     int64  
 18  Śr/mecz   62 non-null     int64  
 19  Dom       62 non-null     int64  
 20  Wyjazd    62 non-null     int64  


In [73]:
mapper = {'M':'Mecze',"Poz":"Pozycja","B":"Biegi","1":"1_miejsce","2":"2_miejsce","3":"3_miejsce",
          "4":"4_miejsce","D":"Defekty","U":"Upadki","W":"Wykluczenia","T":"Tasmy","Pkt.":"Punkty",
          "Śr/bieg":"Srednia_biegowa","Śr/mecz":"Srednia_meczowa","Dom":"Srednia_biegowa_dom",
          "Wyjazd":"Srednia_biegowa_wyjazd"}

In [74]:
df_riders.rename(columns=mapper, inplace=True)

In [75]:
df_riders["Klub"] = "GKM Grudziadz"

In [76]:
df_riders.sample(5)

,Sezon,Pozycja,Zawodnik,Klub,Mecze,Biegi,1_miejsce,2_miejsce,3_miejsce,4_miejsce,...,Upadki,Wykluczenia,Tasmy,Punkty,Bonus,Razem,Srednia_biegowa,Srednia_meczowa,Srednia_biegowa_dom,Srednia_biegowa_wyjazd
35,2021,1,Nicki Pedersen,GKM Grudziadz,14,68,26,23,13,2,...,0,0,0,139,1,140,2059,10692,2200,1947
3,2015,4,Rafał Okoniewski,GKM Grudziadz,14,63,12,16,16,17,...,1,0,0,84,7,91,1444,6000,2000,793
19,2018,2,Przemysław Pawlicki,GKM Grudziadz,14,77,16,35,14,10,...,0,2,0,132,9,141,1831,9429,2000,1675
13,2017,2,Antonio Lindbaeck,GKM Grudziadz,14,64,18,21,17,4,...,1,2,1,113,14,127,1984,8071,2212,1742
4,2015,5,Daniel Jeleniewski,GKM Grudziadz,11,37,4,5,8,16,...,1,2,1,30,9,39,1054,2727,1308,455


In [77]:
df_riders.iloc[:,-4:] = df_riders.iloc[:,-4:].apply(lambda x: x/1000)

In [78]:
df_riders.head(5)

,Sezon,Pozycja,Zawodnik,Klub,Mecze,Biegi,1_miejsce,2_miejsce,3_miejsce,4_miejsce,...,Upadki,Wykluczenia,Tasmy,Punkty,Bonus,Razem,Srednia_biegowa,Srednia_meczowa,Srednia_biegowa_dom,Srednia_biegowa_wyjazd
0,2015,1,Artiom Łaguta,GKM Grudziadz,14,73,29,15,20,4,...,0,0,1,137,7,144,1.973,9.786,2.147,1.821
1,2015,2,Krzysztof Buczkowski,GKM Grudziadz,14,77,16,23,31,6,...,0,0,0,125,13,138,1.792,8.929,2.135,1.475
2,2015,3,Tomasz Gollob,GKM Grudziadz,14,72,14,24,17,14,...,0,0,0,107,9,116,1.611,7.643,2.243,0.943
3,2015,4,Rafał Okoniewski,GKM Grudziadz,14,63,12,16,16,17,...,1,0,0,84,7,91,1.444,6.000,2.000,0.793
4,2015,5,Daniel Jeleniewski,GKM Grudziadz,11,37,4,5,8,16,...,1,2,1,30,9,39,1.054,2.727,1.308,0.455


In [86]:
df_riders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Sezon                   62 non-null     int32  
 1   Pozycja                 62 non-null     int64  
 2   Zawodnik                62 non-null     object 
 3   Klub                    62 non-null     object 
 4   Mecze                   62 non-null     int64  
 5   Biegi                   62 non-null     int64  
 6   1_miejsce               62 non-null     int64  
 7   2_miejsce               62 non-null     int64  
 8   3_miejsce               62 non-null     int64  
 9   4_miejsce               62 non-null     int64  
 10  Defekty                 62 non-null     int64  
 11  Upadki                  62 non-null     int64  
 12  Wykluczenia             62 non-null     int64  
 13  Tasmy                   62 non-null     int64  
 14  Punkty                  62 non-null     int6

# Insert GKM Riders Data into MS SQL Table

In [101]:
# Database connection parameters
server_name = 'DESKTOP-F00PL4B\SQLEXPRESS01'
database_name = 'PortfolioProject'
table_name = 'GKM_Zawodnicy_Wyniki'

# SQLAlchemy engine creation with Windows Authentication
engine = create_engine(f"mssql+pyodbc://@{server_name}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Insert DataFrame into the existing table with column mapping
df_riders.to_sql(name=table_name, con=engine, if_exists='append', index=False)

# Close the database connection
engine.dispose()